In [ ]:
!pip install tensorflow tensorflow-hub
!pip install opencv-python-headless

import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd

In [ ]:
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

In [ ]:
def run_ssd_on_frame(model, frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (300, 300))
    frame_uint8 = frame_resized.astype(np.uint8)
    input_tensor = tf.convert_to_tensor([frame_uint8], dtype=tf.uint8)
    detections = model(input_tensor)
    return detections

class_id_to_name = {
    1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane',
    6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light',
    11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench',
    16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep',
    21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe',
    27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie',
    33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',
    37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
    41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',
    46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
    51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',
    56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza',
    60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant',
    65: 'bed', 67: 'dining table', 70: 'toilet', 72: 'tv',
    73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard',
    77: 'cell phone', 78: 'microwave', 79: 'oven', 80: 'toaster',
    81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock',
    86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier',
    90: 'toothbrush'
}


In [ ]:
min_score_thresh = 0
detections_data = []

cap = cv2.VideoCapture('/content/video/vid1.mp4')  # Update with your video path
frame_count = 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections = run_ssd_on_frame(model, frame)
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int64)
    scores = detections['detection_scores'][0].numpy()

    for score, class_id in zip(scores, classes):
        if score >= min_score_thresh:
            object_name = class_id_to_name.get(class_id, 'Unknown')
            detections_data.append((frame_count, object_name, score))

    frame_count += 1

cap.release()

In [ ]:
df = pd.DataFrame(detections_data, columns=['Frame Number', 'Object', 'Confidence Score'])
df.to_csv('/content/vid1.csv', index=False)